# Codon Frequencies
This notebook parses FASTQ files and calculates the frequency of codons at site 734 in Zika Virus NS5 protein.

FASTQ files for each cell population are specified in `samplesheet.csv`

### Notebook setup
Imports

In [1]:
import gzip
import os.path
import numpy as np
import pandas as pd
import plotnine as p9
import regex
from Bio import SeqIO

# print imports and versions
# copied from https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook
print('\n'.join(f'{m.__name__}=={m.__version__}' \
                for m in globals().values() if getattr(m, '__version__', None)))

numpy==1.19.1
pandas==1.3.2
plotnine==0.8.0
regex==2.5.83


Input data

In [2]:
samplesheet = 'samplesheet.csv'
reference_file = 'ns5.fasta'
stat2_luciferase_file = 'STAT2_luciferase_results.csv'
permitted_codons_file = 'permitted_codons.txt'
codon_lookup_file = 'codon_lookup_table.csv'
codons_file = 'codons_parsed.csv.gz'
codon_counts_file = 'results/codon_counts.csv'

Analysis parameters

In [3]:
codon_start = 2203  # nposition in reference for first codon nt
upstream_length = 15  # length of sequence before codon to search for
downstream_length = 15  # length of sequence after codon to search for
mismatches = 1  # number of mismatches allowed (per upstream/downstream sequence)
nucleotides_allowed = 'ACGT'  # bases permitted in parsed codon

Notebook aesthetics

In [4]:
p9.theme_set(p9.theme_classic())
CBPALETTE_RICH = ['#648FFF', '#FFB000', '#DC267F', '#785EF0', '#FE6100']

### Load data
Load samples

In [5]:
samples = pd.read_csv(samplesheet)
display(samples)

,sample_name,infected,IFN,read,fastq_file
0,WT_NS5_Sample,True,None,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
1,Mut_NS5_Sample,True,None,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
2,Mut_Rnd1_mCherry_neg,True,False,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
3,Mut_Rnd1_mCherry_pos,True,True,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
4,Mut_Rnd2_mCherry_neg,True,False,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
5,Mut_Rnd2_mCherry_pos,True,True,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
6,WT_Plasmid,None,None,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
7,Mut_Plasmid,None,None,R1,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
8,WT_NS5_Sample,True,None,R2,/shared/ngs/illumina/bloom_lab/210917_M04866_0...
9,Mut_NS5_Sample,True,None,R2,/shared/ngs/illumina/bloom_lab/210917_M04866_0...


Load reference sequence

In [6]:
reference = SeqIO.read(reference_file, 'fasta')
print(reference.id)
print(reference.seq)

NS5
CGTGGAGGTGGGACGGGAGAGACTCTGGGAGAGAAGTGGAAAGCTCGTCTGAATCAGATGTCGGCCCTGGAGTTCTACTCTTATAAAAAGTCAGGTATCACTGAAGTGTGTAGAGAGGAGGCTCGCCGTGCCCTCAAGGATGGAGTGGCCACAGGAGGACATGCCGTATCCCGGGGAAGTGCAAAGCTCAGATGGTTGGTGGAGAGAGGATATCTGCAGCCCTATGGGAAGGTTGTTGACCTCGGATGTGGCAGAGGGGGCTGGAGCTATTATGCCGCCACCATCCGCAAAGTGCAGGAGGTGAGAGGATACACAAAGGGAGGTCCCGGTCATGAAGAACCCATGCTGGTGCAAAGCTATGGGTGGAACATAGTTCGTCTCAAGAGTGGAGTGGACGTCTTCCACATGGCGGCTGAGCCGTGTGACACTCTGCTGTGTGACATAGGTGAGTCATCATCTAGTCCTGAAGTGGAAGAGACACGAACACTCAGAGTGCTCTCTATGGTGGGGGACTGGCTTGAAAAAAGACCAGGGGCCTTCTGTATAAAGGTGCTGTGCCCATACACCAGCACTATGATGGAAACCATGGAGCGACTGCAACGTAGGCATGGGGGAGGATTAGTCAGAGTGCCATTGTCTCGCAACTCCACACATGAGATGTACTGGGTCTCTGGGGCAAAGAGCAACATCATAAAAAGTGTGTCCACCACAAGTCAGCTCCTCCTGGGACGCATGGATGGCCCCAGGAGGCCAGTGAAATATGAGGAGGATGTGAACCTCGGCTCGGGTACACGAGCTGTGGCAAGCTGTGCTGAGGCTCCTAACATGAAAATCATCGGCAGGCGCATTGAGAGAATCCGCAATGAACATGCAGAAACATGGTTTCTTGATGAAAACCACCCATACAGGACATGGGCCTACCATGGGAGCTACGAAGCCCCCACGCAAGGATCAGCGTCTTCCCTCGTGAACGGGGTTGTTAGACTCCTGTCAAAG

### Parse codon sequence
If codons file is present, open codon counts.

Otherwise, parse codons from FASTQ files listed in samplesheet.

Open codons file if present

In [ ]:
if os.path.isfile(codons_file):
    print(f'Loading results from {codons_file}')
    results = pd.read_csv(codons_file)
    print('Done.')
else:
    pass

Loading results from codons_parsed.csv.gz


Extract sequences upstream and downstream of codon

In [ ]:
# feature positions in reference sequence
codon_end = codon_start + 2
upstream_start = codon_start - upstream_length
upstream_end = codon_start - 1
downstream_start = codon_end + 1
downstream_end = codon_end + downstream_length

# feature values
# python indexing is not inclusive of first coordinate, so
# all start positions are adjusted by -1
codon_reference = reference.seq[(codon_start-1):codon_end]
upstream = reference.seq[(upstream_start-1):upstream_end]
downstream = reference.seq[(downstream_start-1):downstream_end]

# print
print(f'The codon starts at position {codon_start} and '
      f'ends at position {codon_end} in the reference.')
print(f'The WT codon sequence in the reference is: {codon_reference}\n')
print(f'The upstream search sequence starts at position {upstream_start} '
      f'and ends at position {upstream_end} in the reference.')
print(f'The upstream search sequence is: {upstream}\n')
print(f'The downstream search sequence starts at position {downstream_start} '
      f'and ends at position {downstream_end} in the reference.')
print(f'The downstream search sequence is: {downstream}')

Build regex search

In [ ]:
pattern = (f'(?:{upstream}){{e<={mismatches}}}' +  # match upstream
              f'([{nucleotides_allowed}]{{3}})' +  # match and capture codon
              f'(?:{downstream}){{e<={mismatches}}}')  # match downstream
print(f'The search pattern for will be:\n{pattern}')

Iterate through reads and store codons:

In [ ]:
if os.path.isfile(codons_file):
    pass
else:
    results = list()

    for index, sample in samples.iterrows():
        assert sample['read'] in ['R1', 'R2'], "unrecognized read"
        with gzip.open(sample['fastq_file'], mode='rt') as handle:
            print(f'Parsing data for {sample["sample_name"]} {sample["read"]}')
            print(f'File is:  {sample["fastq_file"]}')
            seqs = SeqIO.parse(handle, 'fastq')
            for sequence in seqs:
                if(sample['read'] == 'R1'):
                    search_seq = sequence.seq
                elif(sample['read'] == 'R2'):
                    search_seq = sequence.seq.reverse_complement()
                result = regex.search(pattern, str(search_seq))
                if result == None:
                    match = 'failed_parse'
                    codon = 'failed_parse'
                else:
                    match = result.group(0)
                    codon = result.group(1)
                results.append((sample['sample_name'],
                                sample['read'],
                                sequence.id,
                                match,
                                codon))
            print('\n')

    results = pd.DataFrame(results)
    results.columns = ['sample_name','read','read_id','match_string','codon']
    print(f'Saving results to {codons_file}')
    results.to_csv(codons_file, index=False)

In [ ]:
display(results)

### Quality control

Plot number of reads with codon successfully parsed per sample

In [ ]:
parsing_qc = (p9.ggplot(results) +
              p9.aes(x='sample_name',
                     fill='codon != "failed_parse"') +
              p9.geom_bar(stat='count') +
              p9.facet_grid('read~') +
              p9.ggtitle('Codon parsing') +
              p9.labs(x='sample',
                      y='reads',
                      fill='codon successfully parsed') +
              p9.theme(figure_size=(0.5*samples['sample_name'].nunique(), 
                                    1.5*samples['read'].nunique()),
                       plot_title=p9.element_text(size=11),
                       axis_title=p9.element_text(size=10),
                       axis_text_x=p9.element_text(rotation=45, hjust=1),
                       legend_position='right',
                       legend_title=p9.element_text(size=10),
                       legend_title_align='center') +
              p9.scale_fill_manual(CBPALETTE_RICH[0:])
              )

display(parsing_qc)

Pair reads by read_id

In [ ]:
results_pairs = (
    results
    .pivot(index=['sample_name','read_id'], columns='read', values='codon')
    .reset_index())

#  Check for missing mate
results_pairs['missing_mate'] = (
    results_pairs['R1'].isnull() | results_pairs['R2'].isnull()
)

#  Check for mismatch
results_pairs['pair_mismatch'] = (
    results_pairs['R1'] != results_pairs['R2']
)

display(results_pairs)

Plot missing mates

In [ ]:
mate_pair_qc = (p9.ggplot(results_pairs) +
              p9.aes(x='sample_name',
                     fill='missing_mate') +
              p9.geom_bar(stat='count') +
              p9.ggtitle('Missing read mate pairs') +
              p9.labs(x='sample',
                      y='reads',
                      fill='missing mate pair') +
              p9.theme(figure_size=(0.5*samples['sample_name'].nunique(), 
                                    1.5*samples['read'].nunique()),
                       plot_title=p9.element_text(size=11),
                       axis_title=p9.element_text(size=10),
                       axis_text_x=p9.element_text(rotation=45, hjust=1),
                       legend_position='right',
                       legend_title=p9.element_text(size=10),
                       legend_title_align='center') +
              p9.scale_fill_manual(CBPALETTE_RICH[0:])
              )

display(mate_pair_qc)

Plot mismatched codon values from paired reads

In [ ]:
mismatched_pairs_qc = (p9.ggplot(results_pairs) +
              p9.aes(x='sample_name',
                     fill='pair_mismatch') +
              p9.geom_bar(stat='count') +
              p9.ggtitle('Mismatched codon in read mate pairs') +
              p9.labs(x='sample',
                      y='reads',
                      fill='mismatched mate pair') +
              p9.theme(figure_size=(0.5*samples['sample_name'].nunique(), 
                                    1.5*samples['read'].nunique()),
                       plot_title=p9.element_text(size=11),
                       axis_title=p9.element_text(size=10),
                       axis_text_x=p9.element_text(rotation=45, hjust=1),
                       legend_position='right',
                       legend_title=p9.element_text(size=10),
                       legend_title_align='center') +
              p9.scale_fill_manual(CBPALETTE_RICH[0:])
              )

display(mismatched_pairs_qc)

### Error rate
Calculate the error rate for this system. Simply calculate the fraction of reads that are *not WT* for samples expected to be 100% WT.


Label WT codons

In [ ]:
results['wt'] = (results['codon'] == 'GAT')
display(results)

WT vs mutant codons

In [ ]:
wt_counts = (p9.ggplot(results) +
              p9.aes(x='sample_name',
                     fill='wt') +
              p9.geom_bar(stat='count') +
              p9.ggtitle('WT Codon counts') +
              p9.labs(x='sample',
                      y='reads',
                      fill='WT Codon') +
              p9.theme(figure_size=(0.5*samples['sample_name'].nunique(), 
                                    1.5*samples['read'].nunique()),
                       plot_title=p9.element_text(size=11),
                       axis_title=p9.element_text(size=10),
                       axis_text_x=p9.element_text(rotation=45, hjust=1),
                       legend_position='right',
                       legend_title=p9.element_text(size=10),
                       legend_title_align='center') +
              p9.scale_fill_manual(CBPALETTE_RICH[0:])
              )

display(wt_counts)

In [ ]:
wt_freq = (
    results
    .query('sample_name.str.contains("WT")', engine='python')
    .groupby('sample_name')
    ['wt']
    .value_counts(normalize=True)
    .reset_index(name='wt_frac')
)

print('The error rate observed in WT samples is:')
display(wt_freq.query('wt == False'))

### Filter data
Filter the data based on the QC metrics above.

In [ ]:
results_filtered = (
    results_pairs
    .query(
        'missing_mate == False & '
        'pair_mismatch == False & '
        'R1 != "failed_parse" & '
        'R2 != "failed_parse"')
    [['sample_name', 'read_id', 'R1']]
    .rename(columns={'R1': 'codon'})
)

display(results_filtered)

#### Restrict analysis to codons designed to be in library

Only 21 codons (of 64 possible) were designed to be included in the `Mut_Plasmid` library. Restrict the analysis to these codons.

In [ ]:
permitted_codons = pd.read_csv(permitted_codons_file, squeeze=True, header=None).values.tolist()
assert len(permitted_codons) == 21, \
    "Length of codons does not match library size"
display(permitted_codons)

In [ ]:
boolean_filter = results_filtered['codon'].isin(permitted_codons)
results_filtered = results_filtered[boolean_filter]
assert results_filtered['codon'].isin(permitted_codons).all(), \
    "Codons not in original library"
display(results_filtered)

### Plot codon counts

Count codons in each sample

In [ ]:
codon_counts = (
    results_filtered
    .groupby(['sample_name'])
    ['codon']
    .value_counts()
    .reset_index(name='count')
)

counts_per_sample = (
    codon_counts
    .groupby('sample_name')
    ['count']
    .sum()
    .reset_index(name='sample_total')
)

codon_counts = pd.merge(
    left=codon_counts,
    right=counts_per_sample,
    on='sample_name',
    how='left',
    validate='many_to_one'
)

codon_counts['frequency'] = (
    codon_counts['count'] / codon_counts['sample_total']
)

display(codon_counts)

Translate codons to amino acids

In [ ]:
codon_lookup_table = pd.read_csv(codon_lookup_file)
display(codon_lookup_table)

codon_counts = pd.merge(
    left=codon_counts,
    right=codon_lookup_table[['codon', 'letter']],
    on='codon',
    how='left',
    validate='many_to_one'
)

display(codon_counts)

Export filtered codon counts to CSV

In [ ]:
print(f'Saving counts to {codon_counts_file}')
codon_counts.to_csv(codon_counts_file, index=False)

### Plot codon counts and frequencies by sample

In [ ]:
codon_counts_plot = (p9.ggplot(codon_counts) +
                p9.aes(x='codon',
                       y='count') +
                p9.geom_bar(stat='identity') +
                p9.facet_grid('sample_name~', scales='free_y') +
                p9.ggtitle('Codon counts') +
                p9.labs(x='codon',
                        y='reads') +
                p9.theme(figure_size=(9, 5),
                         plot_title=p9.element_text(size=11),
                         axis_title=p9.element_text(size=10),
                         axis_text_x=p9.element_text(size=8, rotation=90),
                         axis_text_y=p9.element_text(size=8),
                         strip_text_y=p9.element_text(size=8,angle=0, ha='left'),
                         strip_background_y=p9.element_text(width=0.22))
               )

display(codon_counts_plot)

Plot relative ratio of codons in mutant plasmid pool and mutant cell population

In [ ]:
mut_codon_freqs = (
    results_filtered
    .query('(codon != "failed_parse") and '
           '(sample_name == "Mut_NS5_Sample" or '
           'sample_name == "Mut_Plasmid")')
    .groupby('sample_name')
    ['codon']
    .value_counts(normalize=True)
    .reset_index(name='frequency')
)

In [ ]:
mut_codon_freqs_plot = (p9.ggplot(mut_codon_freqs) +
                p9.aes(x='codon',
                       y='frequency') +
                p9.geom_bar(stat='identity') +
                p9.facet_grid('sample_name~', scales='free_y') +
                p9.ggtitle('Codon frequencies') +
                p9.labs(x='codon',
                        y='frequency') +
                p9.theme(figure_size=(6, 2),
                         plot_title=p9.element_text(size=10),
                         axis_title=p9.element_text(size=9),
                         axis_text_x=p9.element_text(size=8, rotation=90),
                         axis_text_y=p9.element_text(size=8),
                         strip_text_y=p9.element_text(size=8,angle=0, ha='left'),
                         strip_background_y=p9.element_text(width=0.25))
               )

mut_codon_freqs_plot

### Enrichment calculation function

To calculate enrichment, I will follow some of the general principals used to calculate [DMS amino acid preferences](https://jbloomlab.github.io/dms_tools2/prefs.html#prefs).

Conceptually, I am calculating the ratio of codon frequency in the selected condition compared to codon frequency in the reference condition. I add a pseudocount to accomodate missing codons in one of the two conditions. I take the log2 of the ratio.

**I am not normalizing codon frequency to the WT codon**, because I want to detect and display strong enrichment of the WT codon in the selected condition.

```Enrichment = log2( ((codon_count_selected + pseudocount) / (total_codon_count_selected)) /  
                   ((codon_count_reference + pseudocount) / (total_codon_count_reference)) )```

In [ ]:
def calculate_enrichment(selected_sample,
                         reference_sample="Mut_NS5_Sample",
                         pseudocount=0.1):
    selected_freqs = (codon_counts
                      .query(f'sample_name == "{selected_sample}"')
                      [['codon', 'letter', 'count']])
    selected_freqs['count_pseudo'] = selected_freqs['count'] + pseudocount
    selected_total = float(selected_freqs['count_pseudo'].sum())
    
    reference_freqs = (codon_counts
                       .query(f'sample_name == "{reference_sample}"')
                       [['codon', 'letter', 'count']])
    reference_freqs['count_pseudo'] = reference_freqs['count'] + pseudocount
    reference_total = float(reference_freqs['count_pseudo'].sum())
    
    enrichment_df = pd.merge(
        left=selected_freqs,
        right=reference_freqs,
        on=['codon', 'letter'],
        how='outer',
        validate='one_to_one',
        suffixes=['_selected','_reference'])
    enrichment_df['count_pseudo_selected'] = (enrichment_df['count_pseudo_selected']
                                              .fillna(pseudocount))
    enrichment_df['count_pseudo_reference'] = (enrichment_df['count_pseudo_reference']
                                              .fillna(pseudocount))
    
    enrichment_df['enrichment'] = (
        np.log2((enrichment_df['count_pseudo_selected'] / selected_total) /
                (enrichment_df['count_pseudo_reference'] / reference_total)))
    
    return(enrichment_df)

### Experiment Results

#### Codon tolerance

Calculate the enrichment of codons in the integrated cell popuation ("Mut_NS5_Sample") compared to the plasmid library ("Mut_Plasmid"). This should give an indication of how codons are tolerated when integrated into these cells.

In [ ]:
tolerance = calculate_enrichment(selected_sample='Mut_NS5_Sample',
                                 reference_sample='Mut_Plasmid')
display(tolerance)

tolerance_plot = (p9.ggplot(tolerance) +
                p9.aes(x='codon',
                       y='enrichment') +
                p9.geom_bar(stat='identity') +
                p9.ggtitle('Codon tolerance:\n'
                           f'Mut_NS5_Sample v Mut_Plasmid') +
                p9.labs(x='codon',
                        y='enrichment') +
                p9.theme(figure_size=(6, 2),
                         plot_title=p9.element_text(size=10),
                         axis_title=p9.element_text(size=9),
                         axis_text_x=p9.element_text(size=8, rotation=90),
                         axis_text_y=p9.element_text(size=8))
               )

display(tolerance_plot)

tolerance_file = 'results/tolerance.csv'
print(f'Saving tolerance data to {tolerance_file}')
tolerance.to_csv(tolerance_file, index=False)
print('Done.')

#### IFN antagonism

**Rnd1 IFN- population ("Mut_Rnd1_mCherry_neg")**  
Calculate codon enrichment in the population of NS5-expressing cells that successfully antagonize IFN.  


In [ ]:
selected_pop = "Mut_Rnd1_mCherry_neg"

Rnd1_IFN_neg_enrichment = calculate_enrichment(selected_pop)
display(Rnd1_IFN_neg_enrichment)

Rnd1_IFN_neg_plot = (p9.ggplot(Rnd1_IFN_neg_enrichment) +
                p9.aes(x='codon',
                       y='enrichment') +
                p9.geom_bar(stat='identity') +
                p9.ggtitle('Enrichment:\n'
                           f'{selected_pop} v Mut_NS5_Sample') +
                p9.labs(x='codon',
                        y='enrichment') +
                p9.theme(figure_size=(6, 2),
                         plot_title=p9.element_text(size=10),
                         axis_title=p9.element_text(size=9),
                         axis_text_x=p9.element_text(size=8, rotation=90),
                         axis_text_y=p9.element_text(size=8))
               )

display(Rnd1_IFN_neg_plot)

Rnd1_IFN_neg_file = 'results/enrichment_Rnd1_IFN_neg.csv'
print(f'Saving tolerance data to {Rnd1_IFN_neg_file}')
tolerance.to_csv(Rnd1_IFN_neg_file, index=False)
print('Done.')

**Rnd2 IFN- population ("Mut_Rnd2_mCherry_neg")**  
Calculate codon enrichment in the population of NS5-expressing cells that successfully antagonize IFN.  

In [ ]:
selected_pop = "Mut_Rnd2_mCherry_neg"

Rnd2_IFN_neg_enrichment = calculate_enrichment(selected_pop)
display(Rnd2_IFN_neg_enrichment)

Rnd2_IFN_neg_plot = (p9.ggplot(Rnd2_IFN_neg_enrichment) +
                p9.aes(x='codon',
                       y='enrichment') +
                p9.geom_bar(stat='identity') +
                p9.ggtitle('Enrichment:\n'
                           f'{selected_pop} v Mut_NS5_Sample') +
                p9.labs(x='codon',
                        y='enrichment') +
                p9.theme(figure_size=(6, 2),
                         plot_title=p9.element_text(size=10),
                         axis_title=p9.element_text(size=9),
                         axis_text_x=p9.element_text(size=8, rotation=90),
                         axis_text_y=p9.element_text(size=8))
               )

display(Rnd2_IFN_neg_plot)

**Single round vs two rounds of selection**  
Do the results differe if the cell population is sorted once, or sorted and then sorted again?

In [ ]:
corr_two_rounds = pd.merge(
    left=Rnd1_IFN_neg_enrichment[['codon','enrichment']],
    right=Rnd2_IFN_neg_enrichment[['codon','enrichment']],
    on='codon',
    how='outer',
    validate='one_to_one',
    suffixes=['_Rnd1', '_Rnd2']
)
display(corr_two_rounds)

corr_plot = (p9.ggplot(corr_two_rounds) +
              p9.aes(x='enrichment_Rnd1',
                     y='enrichment_Rnd2') +
              p9.geom_point(alpha=0.75) +
              p9.ggtitle('Correlation between 1 and 2 rounds of flow sorting') +
              p9.labs(x='1 round of flow sorting',
                      y='2 rounds of flow sorting') +
              p9.theme(figure_size=(3,3),
                       plot_title=p9.element_text(size=10),
                       axis_title=p9.element_text(size=9),
                       axis_text_x=p9.element_text(rotation=45, hjust=1),
                       legend_position='right',
                       legend_title=p9.element_text(size=10),
                       legend_title_align='center')
              )

display(corr_plot)

**Failed agonist codons  
Rnd1 IFN+ population ("Mut_Rnd1_mCherry_pos")**  
What does the population that expresses NS5 mutants but still expresses IFN look like?  
Calculate codon enrichment in the population of NS5-expressing cells that **fail to antagonize IFN**.  

In [ ]:
selected_pop = "Mut_Rnd1_mCherry_pos"

Rnd1_IFN_pos_enrichment = calculate_enrichment(selected_pop)
display(Rnd1_IFN_pos_enrichment)

Rnd1_IFN_pos_plot = (p9.ggplot(Rnd1_IFN_pos_enrichment) +
                p9.aes(x='codon',
                       y='enrichment') +
                p9.geom_bar(stat='identity') +
                p9.ggtitle('Enrichment:\n'
                           f'{selected_pop} v Mut_NS5_Sample') +
                p9.labs(x='codon',
                        y='enrichment') +
                p9.theme(figure_size=(6, 2),
                         plot_title=p9.element_text(size=10),
                         axis_title=p9.element_text(size=9),
                         axis_text_x=p9.element_text(size=8, rotation=90),
                         axis_text_y=p9.element_text(size=8))
               )

display(Rnd1_IFN_pos_plot)

**Correlation between IFN+ and IFN- populations**  
Are certain codons enriched through the act of sorting and passaging cells, independent of IFN antagonism?

In [ ]:
Rnd1_pos_neg = pd.merge(
    left=Rnd1_IFN_neg_enrichment[['codon','enrichment']],
    right=Rnd1_IFN_pos_enrichment[['codon','enrichment']],
    on='codon',
    how='outer',
    validate='one_to_one',
    suffixes=['_IFN_neg', '_IFN_pos']
)
display(Rnd1_pos_neg)

Rnd1_pos_neg_plot = (p9.ggplot(Rnd1_pos_neg) +
              p9.aes(x='enrichment_IFN_neg',
                     y='enrichment_IFN_pos') +
              p9.geom_point(alpha=0.75) +
              p9.ggtitle('Correlation between IFN neg and IFN pos Rnd1 populations') +
              p9.labs(x='IFN_neg population',
                      y='IFN_pos population') +
              p9.theme(figure_size=(3,3),
                       plot_title=p9.element_text(size=11),
                       axis_title=p9.element_text(size=10),
                       axis_text_x=p9.element_text(rotation=45, hjust=1),
                       legend_position='right',
                       legend_title=p9.element_text(size=10),
                       legend_title_align='center')
              )

display(Rnd1_pos_neg_plot)

## Comparison to STAT2 antagonism luciferase assay
Each of these amino acids has previously been tested for ability to antagonize STAT2 in a luciferase assay. How do the sequencing-based enrichment scores correlate with the luciferase assay results?

In [ ]:
stat2_results = pd.read_csv(stat2_luciferase_file)
display(stat2_results.sort_values(by='FLUC_RLUC'))

#### Correlation between codon tolerance and STAT2 antagonism  
Does tolerance in the cell population correlate with known antagonism of STAT2?

In [ ]:
tolerance_stat2_corr = (pd.merge(
    left=tolerance[['letter','enrichment']],
    right=stat2_results,
    left_on='letter',
    right_on='amino_acid',
    how='outer',
    validate='one_to_one')
    .drop(columns='letter'))
display(tolerance_stat2_corr.sort_values(by='FLUC_RLUC'))

tolerance_stat2_corr_plot = (
    p9.ggplot(tolerance_stat2_corr) +
    p9.aes(x='enrichment',
           y='FLUC_RLUC') +
    p9.geom_point() +
    p9.ggtitle('Correlation between tolerance and\nSTAT2 antagonism') +
    p9.labs(x='codon tolerance\n(enrichment cell v. plasmid)',
            y='STAT2 antagonism\n(FLUC/RLUC)') +
    p9.scale_y_log10() +
    p9.theme(figure_size=(3,3),
             plot_title=p9.element_text(size=10),
             axis_title=p9.element_text(size=9),
             axis_text_x=p9.element_text(rotation=45, hjust=1),
             legend_position='right',
             legend_title=p9.element_text(size=10),
             legend_title_align='center'))

display(tolerance_stat2_corr_plot)

tolerance_stat2_corr_censored_plot = (
    p9.ggplot(tolerance_stat2_corr
              .query('enrichment > -1')) +
    p9.aes(x='enrichment',
           y='FLUC_RLUC') +
    p9.geom_point() +
    p9.ggtitle('Correlation between tolerance and\nSTAT2 antagonism\n**outliers censored**') +
    p9.labs(x='codon tolerance\n(enrichment cell v. plasmid)',
            y='STAT2 antagonism\n(FLUC/RLUC)') +
    p9.theme(figure_size=(3,3),
             plot_title=p9.element_text(size=10),
             axis_title=p9.element_text(size=9),
             axis_text_x=p9.element_text(rotation=45, hjust=1),
             legend_position='right',
             legend_title=p9.element_text(size=10),
             legend_title_align='center')
              )

display(tolerance_stat2_corr_censored_plot)

#### Correlation between IFN- enrichment and STAT2 antagonism
How does enrichment in the IFN- cell population correlate with known antagonism of STAT2?

In [ ]:
Rnd1_IFN_neg_stat2_corr = (pd.merge(
    left=Rnd1_IFN_neg_enrichment[['letter','enrichment']],
    right=stat2_results,
    left_on='letter',
    right_on='amino_acid',
    how='outer',
    validate='one_to_one')
    .drop(columns='letter'))
display(Rnd1_IFN_neg_stat2_corr.sort_values(by='FLUC_RLUC'))

Rnd1_IFN_neg_stat2_corr_plot = (
    p9.ggplot(Rnd1_IFN_neg_stat2_corr) +
    p9.aes(x='enrichment',
           y='FLUC_RLUC') +
    p9.geom_point() +
    p9.ggtitle('Correlation between IFN- enrichment and\nSTAT2 antagonism') +
    p9.labs(x='IFN- enrichment',
            y='STAT2 antagonism\n(FLUC/RLUC)') +
    p9.scale_y_log10() +
    p9.theme(figure_size=(3,3),
             plot_title=p9.element_text(size=10),
             axis_title=p9.element_text(size=9),
             axis_text_x=p9.element_text(rotation=45, hjust=1),
             legend_position='right',
             legend_title=p9.element_text(size=10),
             legend_title_align='center'))

display(Rnd1_IFN_neg_stat2_corr_plot)

Rnd1_IFN_neg_stat2_corr_censored_plot = (
    p9.ggplot(Rnd1_IFN_neg_stat2_corr
              .query('enrichment < 1')) +
    p9.aes(x='enrichment',
           y='FLUC_RLUC') +
    p9.geom_point() +
    p9.ggtitle('Correlation between IFN- enrichment and\nSTAT2 antagonism\n**outliers censored**') +
    p9.labs(x='IFN- enrichment',
            y='STAT2 antagonism\n(FLUC/RLUC)') +
    p9.theme(figure_size=(3,3),
             plot_title=p9.element_text(size=10),
             axis_title=p9.element_text(size=9),
             axis_text_x=p9.element_text(rotation=45, hjust=1),
             legend_position='right',
             legend_title=p9.element_text(size=10),
             legend_title_align='center')
              )

display(Rnd1_IFN_neg_stat2_corr_censored_plot)